In [3]:
import os,shutil

original_dataset_dir = 'datasets/cats_and_dogs/train'

base_dir = 'datasets/cats_and_dogs_small'
os.mkdir(base_dir)


In [4]:
"train, validation, test 디렉토리 만들기"
train_dir = os.path.join(base_dir,'train')
os.mkdir(train_dir)
validation_dir = os.path.join(base_dir, 'validation')
os.mkdir(validation_dir)
test_dir = os.path.join(base_dir, 'test')
os.mkdir(test_dir)

"train, validation, test 각 디렉토리에 cats 폴더와 dogs 폴더 추가"
train_cats_dir = os.path.join(train_dir, 'cats')
os.mkdir(train_cats_dir)
train_dogs_dir = os.path.join(train_dir,'dogs')
os.mkdir(train_dogs_dir)

validation_cats_dir = os.path.join(validation_dir, 'cats')
os.mkdir(validation_cats_dir)
validation_dogs_dir = os.path.join(validation_dir,'dogs')
os.mkdir(validation_dogs_dir)

test_cats_dir = os.path.join(test_dir, 'cats')
os.mkdir(test_cats_dir)
test_dogs_dir = os.path.join(test_dir,'dogs')
os.mkdir(test_dogs_dir)



In [6]:
"처음 1000개를 훈련용 고양이 폴더에 복사"
fnames = ['cat.{}.jpg'.format(i) for i in range(1000)]
for fname in fnames :
    src = os.path.join(original_dataset_dir, fname)
    dst = os.path.join(train_cats_dir, fname)
    shutil.copyfile(src, dst)
    
"다음 500개를 검증용 고양이 폴더에 복사"
fnames = ['cat.{}.jpg'.format(i) for i in range(1000,1500)]
for fname in fnames :
    src = os.path.join(original_dataset_dir, fname)
    dst = os.path.join(validation_cats_dir, fname)
    shutil.copyfile(src, dst)

"다음 500개를 테스트용 고양이 폴더에 복사"
fnames = ['cat.{}.jpg'.format(i) for i in range(1500,2000)]
for fname in fnames :
    src = os.path.join(original_dataset_dir, fname)
    dst = os.path.join(test_cats_dir, fname)
    shutil.copyfile(src, dst)

    

"처음 1000개를 훈련용 강아지 폴더에 복사"
fnames = ['dog.{}.jpg'.format(i) for i in range(1000)]
for fname in fnames :
    src = os.path.join(original_dataset_dir, fname)
    dst = os.path.join(train_dogs_dir, fname)
    shutil.copyfile(src, dst)

"다음 500개를 검증용 강아지 폴더에 복사"
fnames = ['dog.{}.jpg'.format(i) for i in range(1000,1500)]
for fname in fnames :
    src = os.path.join(original_dataset_dir, fname)
    dst = os.path.join(validation_dogs_dir, fname)
    shutil.copyfile(src, dst)

"다음 500개를 테스트용 강아지 폴더에 복사"
fnames = ['dog.{}.jpg'.format(i) for i in range(1500,2000)]
for fname in fnames :
    src = os.path.join(original_dataset_dir, fname)
    dst = os.path.join(test_dogs_dir, fname)
    shutil.copyfile(src, dst)


In [9]:
print('훈련용 고양이 이미지 수 : ',len(os.listdir(train_cats_dir)))
print('검증용 고양이 이미지 수 : ',len(os.listdir(validation_cats_dir)))
print('테스트용 고양이 이미지 수 : ',len(os.listdir(test_cats_dir)))
print()
print('훈련용 강아지 이미지 수 : ',len(os.listdir(train_dogs_dir)))
print('검증용 강아지 이미지 수 : ',len(os.listdir(validation_dogs_dir)))
print('테스트용 강아지 이미지 수 : ',len(os.listdir(test_dogs_dir)))

훈련용 고양이 이미지 수 :  1000
검증용 고양이 이미지 수 :  500
테스트용 고양이 이미지 수 :  500

훈련용 강아지 이미지 수 :  1000
검증용 강아지 이미지 수 :  500
테스트용 강아지 이미지 수 :  500


In [13]:
from keras import layers
from keras import models

model = models.Sequential()

model.add(layers.Conv2D(32,(3,3),activation = 'relu', input_shape = (150, 150, 3)))
model.add(layers.MaxPooling2D(2,2))
model.add(layers.Conv2D(64,(3,3),activation = 'relu'))
model.add(layers.MaxPooling2D(2,2))
model.add(layers.Conv2D(128,(3,3),activation = 'relu'))
model.add(layers.MaxPooling2D(2,2))
model.add(layers.Conv2D(128,(3,3),activation = 'relu'))
model.add(layers.MaxPooling2D(2,2))

model.add(layers.Flatten())
model.add(layers.Dense(512,activation = 'relu'))
model.add(layers.Dense(1, activation = 'sigmoid'))


In [14]:
model.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_10 (Conv2D)           (None, 148, 148, 32)      896       
_________________________________________________________________
max_pooling2d_9 (MaxPooling2 (None, 74, 74, 32)        0         
_________________________________________________________________
conv2d_11 (Conv2D)           (None, 72, 72, 64)        18496     
_________________________________________________________________
max_pooling2d_10 (MaxPooling (None, 36, 36, 64)        0         
_________________________________________________________________
conv2d_12 (Conv2D)           (None, 34, 34, 128)       73856     
_________________________________________________________________
max_pooling2d_11 (MaxPooling (None, 17, 17, 128)       0         
_________________________________________________________________
conv2d_13 (Conv2D)           (None, 15, 15, 128)      

In [15]:
from keras import optimizers

model.compile(loss = 'binary_crossentropy',
             optimizer = optimizers.RMSprop(lr = 1e-4),
             metrics = ['acc'])

In [18]:
from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale = 1./255)
test_datagen = ImageDataGenerator(rescale = 1./255)

train_generator = train_datagen.flow_from_directory(train_dir, target_size = (150,150), batch_size = 20, class_mode = 'binary')

validation_generator = test_datagen.flow_from_directory(validation_dir, target_size = (150,150), batch_size = 20, class_mode = 'binary')


Found 2000 images belonging to 2 classes.
Found 1000 images belonging to 2 classes.
